In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [12]:
# Load pictures and labels:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, Reshape, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

def load_pictures(link):
    with open(link, 'rb') as f:
        f.read(16)
        buffer = f.read()
        pictures = np.frombuffer(buffer, dtype=np.uint8).reshape(-1, 28, 28)
    return pictures

def load_labels(link):
    with open(link, 'rb') as f:
        f.read(8)
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

train_images = load_pictures('/content/drive/MyDrive/MÁSTER/IC/P1/Train/train-images.idx3-ubyte')
train_labels = load_labels('/content/drive/MyDrive/MÁSTER/IC/P1/Train/train-labels.idx1-ubyte')

evaluation_images = load_pictures('/content/drive/MyDrive/MÁSTER/IC/P1/Evaluation/t10k-images.idx3-ubyte')
evaluation_labels = load_labels('/content/drive/MyDrive/MÁSTER/IC/P1/Evaluation/t10k-labels.idx1-ubyte')

train_images = train_images / 255.0
evaluation_images = evaluation_images / 255.0

train_images = train_images.reshape(-1, 28, 28, 1)
evaluation_images = evaluation_images.reshape(-1, 28, 28, 1)

train_labels = to_categorical(train_labels)
evaluation_labels = to_categorical(evaluation_labels)

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.2,
    horizontal_flip=False
)
datagen.fit(train_images)

In [5]:
# Autoencoder
input_img = Input(shape=(28, 28, 1))

x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [6]:
# Training autoencoder
autoencoder.fit(train_images, train_images, epochs=20, batch_size=128, shuffle=True, validation_data=(evaluation_images, evaluation_images))

Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - loss: 0.2190 - val_loss: 0.6857
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0687 - val_loss: 0.0659
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.0663 - val_loss: 0.0654
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - loss: 0.0652 - val_loss: 0.0644
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0642 - val_loss: 0.0632
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.0636 - val_loss: 0.0629
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - loss: 0.0632 - val_loss: 0.0632
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 0.0628 - val_loss: 0.0624
Epoch 9/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 0.0627 - val_loss: 0.0622
Epoch 10/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 0.0623 - val_loss: 0.0619
Epoch 11/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 0.0621 - val_loss: 0.0625
Epoch 12/20
469/469 ━━━━━━━━━━━━

In [7]:
# Neural network:
encoder = Model(input_img, encoded)
encoded_train_images = encoder.predict(train_images)
encoded_evaluation_images = encoder.predict(evaluation_images)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Training neural network
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.fit(
    datagen.flow(train_images, train_labels, batch_size=128),
    epochs=100,
    validation_data=(evaluation_images, evaluation_labels),
    callbacks=[reduce_lr, early_stopping],
    verbose=1
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


469/469 ━━━━━━━━━━━━━━━━━━━━ 39s 64ms/step - accuracy: 0.7352 - loss: 0.8997 - val_accuracy: 0.8409 - val_loss: 0.4814 - learning_rate: 0.0010
Epoch 2/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 30s 53ms/step - accuracy: 0.9541 - loss: 0.1518 - val_accuracy: 0.9893 - val_loss: 0.0300 - learning_rate: 0.0010
Epoch 3/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 53ms/step - accuracy: 0.9683 - loss: 0.1052 - val_accuracy: 0.9915 - val_loss: 0.0235 - learning_rate: 0.0010
Epoch 4/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 25s 54ms/step - accuracy: 0.9739 - loss: 0.0850 - val_accuracy: 0.9927 - val_loss: 0.0226 - learning_rate: 0.0010
Epoch 5/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 24s 51ms/step - accuracy: 0.9776 - loss: 0.0726 - val_accuracy: 0.9931 - val_loss: 0.0216 - learning_rate: 0.0010
Epoch 6/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 42s 53ms/step - accuracy: 0.9798 - loss: 0.0654 - val_accuracy: 0.9939 - val_loss: 0.0202 - learning_rate: 0.0010
Epoch 7/100
469/469 ━━━━━━━━━━━━━━━━━━━━ 26s 55ms/step - accuracy: 0.9828 - loss: 

In [13]:
# Train set:
_, train_accuracy = model.evaluate(train_images, train_labels, verbose=0)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")

# Evaluate set:
_, evaluation_accuracy = model.evaluate(evaluation_images, evaluation_labels, verbose=0)
print(f"Evaluation Accuracy: {evaluation_accuracy * 100:.2f}%")


Train Accuracy: 99.84%
Evaluation Accuracy: 99.70%


In [14]:
evaluation_labels = np.argmax(evaluation_labels, axis=1)
sequence_result = ''.join(map(str, evaluation_labels))

print("Secuencia de etiquetas del conjunto de prueba:")
print(sequence_result)

Secuencia de etiquetas del conjunto de prueba:
72104149590690159734966540740131347271211742351244635560419578937464307029173297762784736136931417696054992194873974449254767905856657810164673171820299551560344654654514472327181818508925011109031642361113952945939036557227128417338879224159872304424195772826857791818030199418212975926415829204002847124027433003196525929304207112153397865613810513155618517946225065637208854114033761621928619525442838245031775797192142920491481845988376003026649333239126805666388275896184125919754089910523789406395213136574226326548971303831934464218254884002327708744796909804606354833933378082170654380963809968685786024022319751084626793298229273591802052137671258037240918677434919517397691378336728585114431077079448554082108450406173267269314625420621734105431174994840245116471942415538314568941538032512834408833173596326136072171424217961124817748073131077035527669283522560829288887493066321322930057814460291474739884712122323239174035586326766327811756